# 2D Heat Convection

In this notebook, we will solve a 2D heat convection problem using the finite difference method. The problem involves simulating the transport of heat in a fluid due to convection. This is a fundamental problem in fluid dynamics and heat transfer, with applications in many engineering fields.

## Governing Equations

The governing equation for 2D heat convection is the 2D convection-diffusion equation, which can be written as:

$$\frac{\partial T}{\partial t} + u \frac{\partial T}{\partial x} + v \frac{\partial T}{\partial y} = \alpha \left(\frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2}\right)$$

where:
- $T$ is the temperature,
- $t$ is the time,
- $x$ and $y$ are the spatial coordinates,
- $u$ and $v$ are the velocities in the $x$ and $y$ directions, respectively, and
- $\alpha$ is the thermal diffusivity.

This equation represents the balance of heat carried by the fluid flow (the convective term) and heat transferred due to temperature gradients (the diffusive term).

## Numerical Scheme

We will discretize this equation using the finite difference method. The time derivative will be discretized using a forward difference, the spatial derivatives will be discretized using central differences, and the diffusive term will be discretized using a second-order central difference. This leads to the following discretized equation:

$$\frac{T_{i,j}^{n+1} - T_{i,j}^n}{\Delta t} + u \frac{T_{i+1,j}^n - T_{i-1,j}^n}{2\Delta x} + v \frac{T_{i,j+1}^n - T_{i,j-1}^n}{2\Delta y} = \alpha \left(\frac{T_{i+1,j}^n - 2T_{i,j}^n + T_{i-1,j}^n}{\Delta x^2} + \frac{T_{i,j+1}^n - 2T_{i,j}^n + T_{i,j-1}^n}{\Delta y^2}\right)$$

where $\Delta t$ is the time step, $\Delta x$ and $\Delta y$ are the spatial steps, and the superscript $n$ denotes the time level.

## Boundary Conditions

The boundary conditions for this problem will be specified as constant temperature values at the boundaries of the domain. These can be implemented in the numerical scheme by simply setting the temperature values at the boundary points to the specified values at each time step.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Domain size and spacing
Lx = 1.0
Ly = 1.0
nx = 41
ny = 41
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

# Time step and end time
dt = 0.001
T_end = 0.5
nt = int(T_end / dt) + 1

# Initialize the temperature field
T = np.zeros((ny, nx))

# Set the velocities
u = 1.0
v = 1.0

# Set the thermal diffusivity
alpha = 0.05

# Time-stepping loop
for n in range(nt):
    Tn = T.copy()
    T[1:-1, 1:-1] = (Tn[1:-1, 1:-1] -
                     dt * u * (Tn[1:-1, 2:] - Tn[1:-1, :-2]) / (2 * dx) -
                     dt * v * (Tn[2:, 1:-1] - Tn[:-2, 1:-1]) / (2 * dy) +
                     alpha * dt * (Tn[1:-1, 2:] - 2 * Tn[1:-1, 1:-1] + Tn[1:-1, :-2]) / dx**2 +
                     alpha * dt * (Tn[2:, 1:-1] - 2 * Tn[1:-1, 1:-1] + Tn[:-2, 1:-1]) / dy**2)

    # Apply the boundary conditions
    T[0, :] = 1.0
    T[-1, :] = 1.0
    T[:, 0] = 1.0
    T[:, -1] = 1.0

    # Plot the temperature field at certain time steps
    if n % 50 == 0:
        plt.figure(figsize=(7, 5))
        plt.contourf(T, cmap='hot', levels=np.linspace(np.min(T), np.max(T), num=100))
        plt.title('Temperature at time step {}'.format(n))
        plt.colorbar()
        plt.show()
        

## Convergence and Error Analysis

In numerical simulations, it's important to ensure that the solution is converging to the correct answer. One way to check this is to compute the solution for different grid resolutions and see if the solution is becoming more accurate as the grid is refined.

In this case, since we don't have an analytical solution to compare with, we will use the method of 'observed order of convergence' (OOC). The idea is to compute the solution for different grid resolutions, compute the difference between the solutions at successive resolutions, and see if this difference is decreasing as the grid is refined.

We will compute the L2 norm of the difference between the solutions at successive resolutions, which is defined as:

$$||T_{h} - T_{2h}|| = \sqrt{\sum_{i,j} (T_{h_{i,j}} - T_{2h_{i,j}})^2}$$

where $T_{h}$ is the solution at the finer resolution and $T_{2h}$ is the solution at the coarser resolution. The L2 norm gives a measure of the 'average' difference between the two solutions over the entire domain.

We expect that, as the grid is refined (i.e., as $h$ decreases), the L2 norm of the difference between the solutions at successive resolutions should also decrease. If it does, this indicates that the solution is converging. Furthermore, for a second-order method (which the finite difference method is), we expect that the L2 norm should decrease by a factor of 4 (approximately) each time the grid resolution is doubled. This is because the error of a second-order method is proportional to $h^2$, so if $h$ is halved, the error should decrease by a factor of $(1/2)^2 = 1/4$.

Let's implement this convergence check in the code.